In [1]:
import herringbone as hb

All initialization tests passed.
imported herringbone without any errors :)


In [ ]:

# create an MDP
map_names = ["example", "easy", "danger_holes", "double_fish", "wall_of_death"]
selected_map_id = 0

state_path = "herringbone/env_core/config/state_config.json"
map_path = f"herringbone/env_core/maps/{map_names[1]}.csv"
action_path = "herringbone/env_core/config/action_config.json"

demo_mdp = hb.MDP(state_path, map_path, action_path, seed=42)


In [3]:
random_policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)
episode = hb.Episode(mdp=demo_mdp, policy=random_policy, max_depth=1000)

In [4]:
# Add the render function for previewing a Value Function (hate this!)
def preview_V(mdp, learned_V):
    
    states_2d = mdp.get_board().states
    x = len(states_2d)
    y = len(states_2d[0])
    
    v_values =  ['%.2f' % v for v in list(learned_V.values())]

    two_d_list = [v_values[i * y:(i + 1) * y] for i in range(x)]

    # Print 2D list
    for row in two_d_list:
        print(row)

In [5]:
policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)  # defaults to uniform
print(policy)

╔═════════╦═════════╦═════════╗
║ ←/→/↑/↓ ║ ←/→/↑/↓ ║ ←/→/↑/↓ ║
╚═════════╩═════════╩═════════╝


In [6]:
episode = hb.Episode(mdp=demo_mdp, policy=policy, max_depth=1000)

In [7]:
episode.run("sar")

t: 0 | S: [0, 0], R: None, A: None
t: 1 | S: [0, 0], R: -1, A: ←
t: 2 | S: [0, 0], R: -1, A: →
t: 3 | S: [0, 0], R: -1, A: ←
t: 4 | S: [0, 0], R: -1, A: ←
t: 5 | S: [0, 0], R: -1, A: ←
t: 6 | S: [0, 0], R: -1, A: →
t: 7 | S: [0, 0], R: -1, A: ←
t: 8 | S: [0, 1], R: -1, A: ↓
t: 9 | S: [0, 0], R: -1, A: ↑
t: 10 | S: [0, 0], R: -1, A: ←
t: 11 | S: [0, 0], R: -1, A: →
t: 12 | S: [0, 0], R: -1, A: ↑
t: 13 | S: [0, 0], R: -1, A: →
t: 14 | S: [0, 0], R: -1, A: ↑
t: 15 | S: [0, 1], R: -1, A: ↓
t: 16 | S: [0, 0], R: -1, A: ↑
t: 17 | S: [0, 0], R: -1, A: ←
t: 18 | S: [0, 0], R: -1, A: ←
t: 19 | S: [0, 1], R: -1, A: ↓
t: 20 | S: [0, 0], R: -1, A: ↑
t: 21 | S: [0, 0], R: -1, A: ↑
t: 22 | S: [0, 0], R: -1, A: ←
t: 23 | S: [0, 0], R: -1, A: →
t: 24 | S: [0, 0], R: -1, A: ←
t: 25 | S: [0, 0], R: -1, A: ←
t: 26 | S: [0, 1], R: -1, A: ↓
t: 27 | S: [0, 0], R: -1, A: ↑
t: 28 | S: [0, 0], R: -1, A: ↑
t: 29 | S: [0, 0], R: -1, A: ↑
t: 30 | S: [0, 0], R: -1, A: ←
t: 31 | S: [0, 0], R: -1, A: →
t: 32 | S: [0

In [ ]:
DISCOUNT = 0.9

In [ ]:
N = 1000
mc_predictor = hb.MonteCarloPredictor(demo_mdp, discount=DISCOUNT)
mc_predictor.evaluate_policy(policy, n_samples=1000)

In [10]:
preview_V(demo_mdp, mc_predictor.value_functions)

['-0.49', '1.05', '0.00']


In [ ]:
mc_control = hb.MonteCarloController(demo_mdp, discount=DISCOUNT, epsilon=0.1, start_coords=(0,0))
mc_control.train(n_episodes=100)
trained_policy = mc_control.policy

In [14]:
print(trained_policy.get_policy())
print(trained_policy)

{[0, 0]: {←: 0.025, →: 0.025, ↑: 0.025, ↓: 0.925}, [0, 1]: {←: 0.03830439806468894, →: 0.03155100690986328, ↑: 0.02462878097523716, ↓: 0.9055158140502105}, [0, 2]: {←: 0.25, →: 0.25, ↑: 0.25, ↓: 0.25}}
╔═════════╦═════════╦═════════╗
║    ↓    ║    ↓    ║ ←/→/↑/↓ ║
╚═════════╩═════════╩═════════╝
